<a href="https://colab.research.google.com/github/2348754114579/Image-and-video-enhancer/blob/main/13_April%F0%9F%87%A7%F0%9F%87%A9%F0%9F%87%A7%F0%9F%87%A9%F0%9F%92%A8%20%E0%A6%A1%E0%A6%BE%E0%A6%89%E0%A6%A8%E0%A6%B2%E0%A7%8B%E0%A6%A1%20%E0%A6%87%E0%A6%89%E0%A6%9F%E0%A6%BF%E0%A6%89%E0%A6%AC%20%E0%A6%AD%E0%A6%BF%E0%A6%A1%E0%A6%BF%E0%A6%93%20%E0%A6%9F%E0%A7%8D%E0%A6%B0%E0%A6%BF%E0%A6%AE%20%E0%A6%93%20%E0%A6%B8%E0%A6%82%E0%A6%AF%E0%A7%8B%E0%A6%9C%E0%A6%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required libraries
!pip install -q yt-dlp
!apt-get install ffmpeg -y

import yt_dlp
import ipywidgets as widgets
from IPython.display import display
import os
import re
import subprocess
from IPython.display import HTML
from google.colab import files
import base64
import json

# --- Text Overhead ---
html_overhead = HTML("<center><b>This module created By Gemini AI with prompts by Black Box</b></center>")
display(html_overhead)

# --- UI Elements for 5 Videos ---
video_inputs = []
downloaded_files = [None] * 5
downloaded_segment_files = [f'segment{i+1}.mkv' for i in range(5)]

for i in range(5):
    url_input = widgets.Text(placeholder=f'Enter URL for Video {i+1}', layout=widgets.Layout(width='50%'))
    start_input = widgets.Text(placeholder=f'Start time (V{i+1})', layout=widgets.Layout(width='30%'))
    end_input = widgets.Text(placeholder=f'End time (V{i+1})', layout=widgets.Layout(width='30%'))
    download_segment_button = widgets.Button(description=f'Download Segment {i+1}')
    download_full_button = widgets.Button(description=f'Download Full Video {i+1}')

    video_inputs.append({
        'url': url_input,
        'start': start_input,
        'end': end_input,
        'download_segment': download_segment_button,
        'download_full': download_full_button
    })

# Stitching UI
stitch_button = widgets.Button(description='Stitch All Segments')
progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Stitching:',
    style={'description_width': 'initial'},
    orientation='horizontal'
)
progress_label = widgets.Label(value='Preparing...')
output_label = widgets.Label()
download_links = widgets.VBox()
stitched_file = 'stitched_video.mkv'

# Save/Load UI
save_button = widgets.Button(description='Save Fields')
load_button = widgets.Button(description='Load Fields')
save_load_output = widgets.Label()
fields_file = 'video_fields.json'

def format_time(time_str):
    """Formats time string to HH:MM:SS if needed."""
    if re.match(r'^\d+$', time_str):
        return f'00:00:{int(time_str):02d}'
    elif re.match(r'^\d+:\d+$', time_str):
        minutes, seconds = map(int, time_str.split(':'))
        return f'00:{minutes:02d}:{seconds:02d}'
    elif re.match(r'^\d+:\d+:\d+$', time_str):
        return time_str
    return None

def create_download_link(filename):
    """Creates a download link for a given filename."""
    if os.path.exists(filename):
        with open(filename, "rb") as f:
            b64 = base64.b64encode(f.read()).decode('utf-8')
        payload = f'<a download="{os.path.basename(filename)}" href="data:application/octet-stream;base64,{b64}" target="_blank">{os.path.basename(filename)}</a>'
        return HTML(payload)
    return HTML(f"Download link for {filename} will appear here.")

def download_full_video(i, b):
    global downloaded_files
    url = video_inputs[i]['url'].value
    if not url:
        output_label.value = f'Please enter a URL for Video {i+1}.'
        return
    output_label.value = f'Downloading full video {i+1} from {url}...'
    ydl_opts = {'outtmpl': f'full_video{i+1}.%(ext)s', 'format': 'bestvideo+bestaudio/best'}
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info_dict = ydl.extract_info(url, download=True)
            downloaded_file_ext = info_dict.get('ext')
            downloaded_files[i] = f'full_video{i+1}.{downloaded_file_ext}'
        output_label.value = f'Full video {i+1} downloaded as {downloaded_files[i]}'
    except Exception as e:
        output_label.value = f'Full video {i+1} download failed: {e}'
        downloaded_files[i] = None

def download_segment(i, b):
    global downloaded_files, downloaded_segment_files
    if not downloaded_files[i] or not os.path.exists(downloaded_files[i]):
        output_label.value = f'Please download full video {i+1} first.'
        return
    start_time_str = video_inputs[i]['start'].value.strip()
    end_time_str = video_inputs[i]['end'].value.strip()
    start_formatted = format_time(start_time_str)
    end_formatted = format_time(end_time_str)
    if not start_formatted or not end_formatted:
        output_label.value = f'Invalid time format for Video {i+1}.'
        return
    output_label.value = f'Extracting segment {i+1} from {start_formatted} to {end_formatted}...'
    try:
        command = f'ffmpeg -i "{downloaded_files[i]}" -ss {start_formatted} -to {end_formatted} -map 0 -c copy "{downloaded_segment_files[i]}"'
        os.system(command)
        if os.path.exists(downloaded_segment_files[i]) and os.path.getsize(downloaded_segment_files[i]) > 0:
            output_label.value = f'Segment {i+1} downloaded as {downloaded_segment_files[i]}'
        else:
            output_label.value = f'Segment {i+1} download failed or is empty.'
    except Exception as e:
        output_label.value = f'Segment {i+1} download failed: {e}'

# Function to save fields
def save_fields(b):
    data = {
        'urls': [vi['url'].value for vi in video_inputs],
        'starts': [vi['start'].value for vi in video_inputs],
        'ends': [vi['end'].value for vi in video_inputs]
    }
    try:
        with open(fields_file, 'w') as f:
            json.dump(data, f)
        save_load_output.value = f'Fields saved to {fields_file}'
    except Exception as e:
        save_load_output.value = f'Error saving fields: {e}'

# Function to load fields
def load_fields(b):
    try:
        if os.path.exists(fields_file):
            with open(fields_file, 'r') as f:
                data = json.load(f)
            for i in range(min(len(video_inputs), len(data.get('urls', [])), len(data.get('starts', [])), len(data.get('ends', [])))):
                video_inputs[i]['url'].value = data['urls'][i]
                video_inputs[i]['start'].value = data['starts'][i]
                video_inputs[i]['end'].value = data['ends'][i]
            save_load_output.value = f'Fields loaded from {fields_file}'
        else:
            save_load_output.value = f'File {fields_file} not found.'
    except Exception as e:
        save_load_output.value = f'Error loading fields: {e}'

# Function to stitch all segments with progress
def stitch_segments(b):
    global downloaded_segment_files, stitched_file, download_links
    valid_segments = [f for f in downloaded_segment_files if os.path.exists(f) and os.path.getsize(f) > 0]

    if not valid_segments:
        output_label.value = 'Please ensure at least one video segment is downloaded.'
        return

    output_label.value = 'Preparing to stitch...'
    progress_bar.value = 0
    progress_label.value = 'Preparing...'
    display(progress_bar, progress_label)

    list_file = 'segments_list.txt'
    with open(list_file, 'w') as f:
        for seg in valid_segments:
            f.write(f"file '{seg}'\n")

    try:
        command = f'ffmpeg -f concat -safe 0 -i {list_file} -c copy "{stitched_file}"'
        process = subprocess.Popen(command, shell=True, stderr=subprocess.PIPE, universal_newlines=True)

        total_duration = 0.0

        while True:
            output = process.stderr.readline()
            if output == '' and process.poll() is not None:
                break
            if output:
                if "Duration:" in output:
                    match = re.search(r"Duration: (\d{2}:\d{2}:\d{2}\.\d{2})", output)
                    if match:
                        h, m, s, ms = map(float, re.split(r'[:.]', match.group(1)))
                        total_duration = h * 3600 + m * 60 + s + ms / 100
                        progress_bar.max = int(total_duration)

                if "time=" in output and total_duration > 0:
                    match = re.search(r"time=(\d{2}:\d{2}:\d{2}\.\d{2})", output)
                    if match:
                        h, m, s, ms = map(float, re.split(r'[:.]', match.group(1)))
                        current_time = h * 3600 + m * 60 + s + ms / 100
                        progress = int((current_time / total_duration) * 100)
                        progress_bar.value = min(progress, 100)
                        progress_label.value = f'Stitching: {progress}%'
                elif "Stream mapping" in output or "Output #" in output:
                    progress_label.value = 'Preparing streams...'
                elif "muxing overhead" in output:
                    progress_label.value = 'Finalizing...'
                elif "Finishing stream" in output:
                    progress_label.value = 'Finish...'

        if os.path.exists(stitched_file) and os.path.getsize(stitched_file) > 0:
            output_label.value = f'Video segments stitched successfully! Saved as {stitched_file}'
            download_links.children = (create_download_link(stitched_file),)
        else:
            output_label.value = f'Stitching process completed, but the output file is empty or invalid. Check segment files.'
        os.remove(list_file)
        display(HTML('<script>document.querySelector(".jupyter-widgets-view[data-model-name=\'IntProgressModel\']").remove(); document.querySelector(".jupyter-widgets-view[data-model-name=\'LabelModel\'][data-view-model-id*=\'' + progress_label._model_id + '\']").remove();</script>'))

    except Exception as e:
        output_label.value = f'Error stitching video segments: {e}'
        if 'progress_bar' in locals():
            display(HTML('<script>document.querySelector(".jupyter-widgets-view[data-model-name=\'IntProgressModel\']").remove(); document.querySelector(".jupyter-widgets-view[data-model-name=\'LabelModel\'][data-view-model-id*=\'' + progress_label._model_id + '\']").remove();</script>'))

# Button click handlers
for i in range(5):
    video_inputs[i]['download_full'].on_click(lambda b, index=i: download_full_video(index, b))
    video_inputs[i]['download_segment'].on_click(lambda b, index=i: download_segment(index, b))

stitch_button.on_click(stitch_segments)
save_button.on_click(save_fields)
load_button.on_click(load_fields)

# UI Display
video_uis = []
for i in range(5):
    video_ui = widgets.VBox([
        widgets.Label(f'Video {i+1}:'),
        video_inputs[i]['url'],
        widgets.HBox([video_inputs[i]['start'], video_inputs[i]['end']]),
        widgets.HBox([video_inputs[i]['download_full'], video_inputs[i]['download_segment']])
    ])
    video_uis.append(video_ui)

display(widgets.VBox([
    *video_uis,
    widgets.HBox([save_button, load_button]),
    save_load_output,
    widgets.Label('Stitching:'),
    stitch_button,
    progress_bar,
    progress_label,
    output_label,
    download_links
]))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.2/172.2 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 22.1 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.


[youtube] Extracting URL: https://youtu.be/kBskrYZfhw8?si=er-2aCdxt7rIug2n
[youtube] kBskrYZfhw8: Downloading webpage
[youtube] kBskrYZfhw8: Downloading tv client config
[youtube] kBskrYZfhw8: Downloading player 179bab65-main
[youtube] kBskrYZfhw8: Downloading tv player API JSON
[youtube] kBskrYZfhw8: Downloading ios player API JSON
[youtube] kBskrYZfhw8: Downloading m3u8 information
[info] kBskrYZfhw8: Downloading 1 format(s): 616+251
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 19
[download] Destination: full_video1.f616.mp4
[download] 100% of   18.40MiB in 00:00:01 at 13.66MiB/s                
[download] Destination: full_video1.f251.webm
[download] 100% of    1.45MiB in 00:00:00 at 33.91MiB/s  
[Merger] Merging formats into "full_video1.webm"
Deleting original file full_video1.f251.webm (pass -k to keep)
Deleting original file full_video1.f616.mp4 (pass -k to keep)
[youtube] Extracting URL: https://youtu.be/UNwzQ9plvFI?si=8rCMr99sFSxVvbFz
[youtube] UNwzQ9plvF

IntProgress(value=0, description='Stitching:', style=ProgressStyle(description_width='initial'))

Label(value='Preparing...')